In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import bioformats.omexml as ome
import tifffile
import sys
import argparse
import os
import logging
import tensorflow as tf
from skimage import exposure#, external
from scipy import ndimage, signal, stats
from math import floor, ceil
from flowdec import data as fd_data
from flowdec import psf as fd_psf
from flowdec import restoration as fd_restoration
from xml.etree import cElementTree as ElementTree
#from tensorflow.python.compiler.mlcompute import mlcompute; 
#mlcompute.set_mlc_device(device_name='gpu')

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')


Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:


dat = tifffile.imread(input_file_str)
psf = np.genfromtxt(bead_image_file_str, delimiter=',')
print("Data successfully read in \n")

kernel = np.zeros(kernel_shape) #(51,51,51)) #Note may not work if this size is too big relative to the image
for offset in [0,1]:
    kernel[tuple((np.array(kernel.shape) - offset) // 2)] = 1
#assume bead image at different resolution in z to actual data, and difference is a factor of 3.08 (308nm versus 100nm)
#estimated and stored 3d psf, but assume here diagonal psf. Could adjust this, but already very close to diagonal
kernel = ndimage.gaussian_filter(kernel, sigma=[np.sqrt(psf[2,2])/3.08,np.sqrt(psf[0,0]),np.sqrt(psf[1,1])])

############

data = np.copy(dat) 
res = np.zeros(data.shape)
ndim = 3 #data.ndim 
algo = fd_restoration.RichardsonLucyDeconvolver(ndim, pad_mode='none', pad_min=[16,16,16]).initialize() 
tmp = algo.run(fd_data.Acquisition(data=data[0], kernel=kernel), niter=2)
for tt in range(SizeT):
    if tt%10==0:
        print(str(tt)+' of ' + str(SizeT) + ' complete')
    nonzero = data[tt][np.where(data[tt]>0)]
    bkgd_mode = stats.mode(nonzero)[0][0]
    bkgd_std = stats.tstd(nonzero)
    indz,indy,indx = np.where(data[tt]==0)
    data[tt,indz, indy, indx] = bkgd_mode + bkgd_std*np.random.randn(len(indz))
    # Note that deconvolution initialization is best kept separate from 
    # execution since the "initialize" operation corresponds to creating 
    # a TensorFlow graph, which is a relatively expensive operation and 
    # should not be repeated across iterations if deconvolving more than 
    # one image
    res[tt] = algo.run(fd_data.Acquisition(data=data[tt], kernel=kernel), niter=niter).data

####################
print('Done. Now writing to file\n')
# create numpy array with correct order
if is_single_time_pt:
    img5d = np.expand_dims(np.expand_dims(res, axis=1), axis=0) 
else:
    img5d = np.expand_dims(res, axis=1)
# write file and save OME-XML as description
tifffile.imwrite(output_file_str, img5d, metadata={'axes': dimorder}, description=xml)
print('All finished\n')